This tutorial describes the basics of assigning users to projects, workspaces, and specific tasks in Label Studio Enterprise using the Python SDK.

## Connect to Label Studio Enterprise

Start by connecting to Label Studio Enterprise using the `LabelStudio` client:

In [ ]:
import os
from label_studio_sdk.client import LabelStudio

ls = LabelStudio(base_url=os.getenv('LABEL_STUDIO_URL', 'http://localhost:8080'), api_key=os.getenv('LABEL_STUDIO_API_KEY'))

Get all users

In [ ]:
users = list(ls.users.list())

Get project by ID

In [ ]:
PROJECT_ID = int(os.getenv('LABEL_STUDIO_PROJECT_ID', '248'))
project = ls.projects.get(id=PROJECT_ID)

Get project members

In [ ]:
members = list(ls.organizations.members.list(id=int(os.getenv('LABEL_STUDIO_ORG_ID', '1'))))

## Add user as a project member

In [ ]:
user = users[0]
res = ls.projects.members.bulk.post(id=PROJECT_ID, all_=False, included=[user.id], roles=[{'role': 'AN', 'user_id': user.id}])  # AN = annotator
res

## Change user role

You can change the role of a user in the currently active organization.

In [ ]:
ORG_ID = int(os.getenv('LABEL_STUDIO_ORG_ID', '1'))
res = ls.organizations.members.update(ORG_ID, user_id=user.id, role='RE')  # RE = reviewer
res

## Assign tasks to annotators or reviewers

You can assign tasks to annotators so that they can work on specific tasks. You must be using manual task distribution in your project to assign tasks.

In [ ]:
from label_studio_sdk.projects.assignments import (
    AssignmentsBulkAssignRequestSelectedItemsIncluded,
)

tasks = ls.tasks.list(project=PROJECT_ID, fields='id')
tasks_ids = [t.id for t in tasks]
# choose 'AN' or 'RE'
ls.projects.assignments.bulk_assign(id=PROJECT_ID, type='AN', users=[u.id for u in users], selected_items=AssignmentsBulkAssignRequestSelectedItemsIncluded(all_=False, included=tasks_ids))

You can assign tasks to reviewers so that they can review the annotations for specific tasks.

In [ ]:
ls.projects.assignments.create(project=PROJECT_ID, type='RE', users=[u.id for u in users], selected_items={'all': False, 'included': tasks_ids})

## List all tasks assigned to specific annotators or reviewers

Before you assign new tasks to annotators, you might want to know which or how many tasks an annotator already has to work on. To list all the tasks already assigned to an annotator, run the following code:

In [ ]:
from label_studio_sdk.data_manager import Filters, Column, Operator, Type

filters = Filters.create(Filters.OR, [
    Filters.item(
        Column.annotators,  # or Column.reviewers
        Operator.CONTAINS,
        Type.List,
        Filters.value(user.id)
    )
])
tasks = list(ls.tasks.list(project=PROJECT_ID, query=filters, fields='id'))
print([t.id for t in tasks])

## Remove annotators or reviewers from tasks

To remove annotators from a task, for example to change who is assigned to a specific task, or remove an annotator that no longer works on the project, run the following code:

In [ ]:
tasks = ls.tasks.list(project=PROJECT_ID, fields='id')
tasks_ids = [t.id for t in tasks]
for task_id in tasks_ids:
    ls.projects.assignments.delete(id=PROJECT_ID, type='AN', task_pk=task_id)  # 'AN' or 'RE'